In [1]:
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

In [2]:
model = ChatOpenAI(model="gpt-4o")

In [3]:
from tools import add_employee_learning_status, create_one_mail_draft

In [5]:
employee_agent = create_react_agent(
    model=model,
    tools=[add_employee_learning_status],
    name="employee_agent",
    prompt="You are an employee agent responsible for managing employee learning status records. You have access to tools that can add employee learning status information to the system. Always use one tool at a time and only when necessary."
)

mail_agent = create_react_agent(
    model=model,
    tools=[create_one_mail_draft],
    name="mail_agent",
    prompt="You are a mail agent responsible for creating mail drafts. You have access to tools that can create mail drafts. Always use one tool at a time and only when necessary. You can't create more than one mail draft at a time."
)

# Create supervisor workflow
workflow = create_supervisor(
    [employee_agent, mail_agent],
    model=model,
    output_mode="last_message", # "full_history"
    prompt=(
        "You are a team supervisor managing a employee agent and a mail agent. "
        "For employee learning status records, use employee_agent. "
        "For mail drafts, use mail_agent."
    )
)

In [6]:
# Compile and run
app = workflow.compile() # TODO add memory

In [ ]:
for chunk in app.stream({
    "messages": [
        {
            "role": "user",
            "content": "Can you please remind lucas@mail.com that he has to attend the training on 28/02/2025?"
        }
    ]
}, stream_mode="updates"):
    print(chunk)

In [ ]:
# Invoke with initial state
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Add learning status for employee John Doe: completed Python course on 2023-05-15 with score 95/100"
        }
    ]
})

# Display the result
print("Final result:")
print(result)


In [ ]:
result